In [1]:
import sys

sys.path.append("/mnt/8de37f61-64a8-4a87-8e84-52eef99e123a/thesis/mscemanuelkuhn/code/evaluation/pipeline/")
sys.path.append("/mnt/8de37f61-64a8-4a87-8e84-52eef99e123a/thesis/mscemanuelkuhn/code/")

# from step_4_compute_triplet_accuracy import compute_triplet_accuracy

In [2]:
import sys

sys.path.append("/home/emanuel/Documents/thesis/layoutgmn_reproductions/LayoutGMN-pytorch/")
sys.path.append("/home/emanuel/Documents/thesis/layoutgmn_reproductions/LayoutGMN-pytorch/training_scripts")

from cross_graph_communication_5 import GraphMatchingNet
from graph_encoder_0 import GraphEncoder, MLP
from graph_aggregator_2 import GraphAggregator

from util import get_args

In [3]:
import wandb
import torch


def download_model_weights_from_wandb(pretrained_wandb_model_ref):

    assert wandb.run is not None, "wandb.run is None; wandb.init() must be called before download_model_weights_from_wandb()"

    model_art: wandb.Artifact = wandb.run.use_artifact(pretrained_wandb_model_ref, type="model")

    pretrained_path = model_art.file()

    starting_epoch = min(int(model_art.metadata["epoch"]), int(model_art.logged_by().summary["epoch"]))

    if starting_epoch < model_art.metadata["epoch"]:
        print(f"WARNING: Loaded model has incorrect epoch metadata: {starting_epoch=} < {model_art.metadata['epoch']=}")
        print("Using max epoch of run as starting epoch")

    print(f'Downloaded pretrained model to:  {pretrained_path} (wandb reference: {pretrained_wandb_model_ref})')

    return pretrained_path, starting_epoch
    

def load_pretrained_model(gmn_model, pretrained_path):
    '''
    :param gmn_model: network model
    :param save_dir: path of the dir where the models have been saved
    :param stored_epoch: str, ex: '8'
    '''
    print('Loading pretrained models')
    
    gmn_model_state_dict = torch.load(pretrained_path)

    from collections import OrderedDict

    def remove_module_fromStateDict(model_state_dict, model):
        new_state_dict = OrderedDict()
        for k, v in model_state_dict.items():
            name = k[0:]  # if ran on two GPUs, remove 'module.module.'; else, no change
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)
        return model

    loaded_gmn_model = remove_module_fromStateDict(gmn_model_state_dict, gmn_model)
    print('Finished loading checkpoint')
    return loaded_gmn_model


In [4]:
config = get_args({})

encoder_model =  GraphEncoder(config, node_hidden_sizes=[config.node_geometry_feat_dim, config.node_state_dim],
                              edge_hidden_sizes=[config.edge_feat_dim, int(config.node_state_dim)])

aggregator_model = GraphAggregator(node_hidden_sizes=[config.graph_rep_dim],
          config=config,
          graph_transform_sizes=[config.graph_rep_dim],
          gated=True,
          aggregation_type='sum')


message_net = MLP([2*config.node_state_dim+int(config.node_state_dim), config.edge_feat_dim, int(config.node_state_dim), int(config.node_state_dim)])
reverse_message_net = MLP([2*config.node_state_dim+int(config.node_state_dim), config.edge_feat_dim, int(config.node_state_dim), int(config.node_state_dim)])
node_update_mlp = MLP([2*config.node_state_dim+int(config.node_state_dim), config.node_geometry_feat_dim, int(config.node_state_dim), config.node_state_dim])


gmn_net = GraphMatchingNet(encoder = encoder_model,
               aggregator = aggregator_model,
               message_net = message_net,
               reverse_message_net = reverse_message_net,
               node_update_MLP = node_update_mlp,
               node_state_dim = config.node_state_dim,
               edge_hidden_sizes = [config.edge_feat_dim, config.node_state_dim * 2,
                                    config.node_state_dim * 2],
               node_hidden_sizes = [config.node_geometry_feat_dim, config.node_state_dim * 2],
               n_prop_layers = 5,
               share_prop_params=False,
               #edge_net_init_scale=0.1,
               node_update_type='residual',
               use_reverse_direction=False,
               reverse_dir_param_different=False,
               layer_norm=False,
               similarity='dotproduct')


In [5]:
wandb.init("layoutgmn_compute_triplet_accuracy", resume=True)

pretrained_path, starting_epoch = download_model_weights_from_wandb("layout_gmn/gmn_model_l3s1sr0w:v34")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: emanuel. Use `wandb login --relogin` to force relogin


Downloaded pretrained model to:  ./artifacts/gmn_model_l3s1sr0w:v34/gmn_tmp_model166.pkl (wandb reference: layout_gmn/gmn_model_l3s1sr0w:v34)


In [6]:
loaded_gmn_model = load_pretrained_model(gmn_net, pretrained_path)

Loading pretrained models
Finished loading checkpoint


In [7]:
import pandas as pd

test_triplets_csv = "/media/emanuel/8de37f61-64a8-4a87-8e84-52eef99e123a/thesis/mscemanuelkuhn/code/evaluation/test_triplets/triplets_apn_iou.csv"

test_triplets = pd.read_csv(test_triplets_csv).to_dict(orient="records")

# Triplet inference

In [8]:
config = get_args("")

In [13]:
import pickle

info = pickle.load(open('../training_scripts/layoutgmn_data/FP_box_info_list.pkl', 'rb'))

In [17]:
from inference.inference_module_ui import get_batch_sg_data, data_input_to_gmn, reshape_and_split_tensor, compute_similarity

In [15]:
test_triplets[0]

{'anchor': 68161, 'positive': 40097, 'negative': 66563}

In [16]:
data_getter = get_batch_sg_data(68161, [40097, 66563], 4,
                                         config, info)

data_getter

# sg_data_a = data['sg_data_a']
# sg_data_fp_1 = data['sg_data_p']
# sg_data_fp_2 = data['sg_data_n']

# assert len(sg_data_a) == len(sg_data_fp_1) == len(sg_data_fp_2)

# assert len(sg_data_a) > 0, f"sg_data_a is empty for {query_fp=}; {sg_data_a=}"

# Graph_Data_dict = data_input_to_gmn(config, self.device,
#                                     sg_data_a, sg_data_fp_1, sg_data_fp_2).quadruples()
# graph_vecs = self.loaded_gmn_model(**Graph_Data_dict)
# x1, y, x2, z = reshape_and_split_tensor(graph_vecs, 4)

# sim_1 = compute_similarity(self.config, x1, y) # these are now list of tensors \n;
# # previously there was torch.mean at the start of the RHS
# sim_2 = compute_similarity(self.config, x2, z) # list of tensors

# sim_val_list_1.append(-(sim_1.cpu().detach().numpy()))
# sim_val_list_2.append(-(sim_2.cpu().detach().numpy()))


In [104]:
def compute_for_triplet(triplet):
    triplet = {key: str(value) for key, value in triplet.items()}

    queries = [data_getter.get_graph_data_by_id(triplet["anchor"])]
    pos = [data_getter.get_graph_data_by_id(triplet["positive"])]
    negs = [data_getter.get_graph_data_by_id(triplet["negative"])]

    data_getter.batch_sg(queries)

    data = {}

    data['sg_data_a'] = data_getter.batch_sg(queries)  # , max_box_len_a)
    data['sg_data_p'] = data_getter.batch_sg(pos)  # , max_box_len_p)
    data['sg_data_n'] = data_getter.batch_sg(negs)  # , max_box_len_n)

    assert len(data['sg_data_a']) == len(data['sg_data_p']) == len(data['sg_data_n'])
    assert len(data['sg_data_a']) > 0, f"sg_data_a is empty for; {data['sg_data_a']=}"


    graph_data_dict = data_input_to_gmn(config, "cpu",
                                        data['sg_data_a'], data['sg_data_p'], data['sg_data_n']).quadruples()

    with torch.no_grad():

        graph_vecs = loaded_gmn_model(**graph_data_dict)
        x1, y, x2, z = reshape_and_split_tensor(graph_vecs, 4)

        sim_1 = compute_similarity(config, x1, y) # these are now list of tensors \n;
        # previously there was torch.mean at the start of the RHS
        sim_2 = compute_similarity(config, x2, z) # list of tensors
    
    return sim_1.item(), sim_2.item()

In [109]:
test_triplets_csv

'/media/emanuel/8de37f61-64a8-4a87-8e84-52eef99e123a/thesis/mscemanuelkuhn/code/evaluation/test_triplets/triplets_apn_iou.csv'

In [110]:
starting_epoch

166

In [108]:
len(test_triplets)

1000

# Compute triplet accuracy

In [107]:
correct = 0
total = 0

for triplet in test_triplets:
    try:
        sim_1, sim_2 = compute_for_triplet(triplet)

        # print(f"{sim_1=}, {sim_2=}, {sim_1 > sim_2=}")

        if sim_1 > sim_2:
            correct += 1
        
        total += 1
    except:
        # print("failed")
        continue

correct, total, correct / total

(705, 945, 0.746031746031746)

The triplet accuracy of this model that was trained for 166 epochs is 0.746, which is a lot lower than reported in the layoutgmn paper (97.54 for the IoU based triplets, which I think are sampled in a similar fashion to the triplets used in this notebook.)